<a href="https://colab.research.google.com/github/oscarcasas21/Mask_RCNN_Multispectral_Bands/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MASK RCNN Tailings